In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

# Set the device to be used for training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the transform to be applied to the input data
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

# Load the training and testing data
trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True, num_workers=2)
testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False, num_workers=2)

# Define the neural network architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 4 * 4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(nn.functional.relu(self.conv1(x)))
        x = self.pool(nn.functional.relu(self.conv2(x)))
        x = x.view(-1, 16 * 4 * 4)
        x = nn.functional.relu(self.fc1(x))
        x = nn.functional.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Create an instance of the neural network
net = Net().to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)

# Train the network
for epoch in range(10):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # Get the inputs and labels
        inputs, labels = data[0].to(device), data[1].to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # Print statistics
        running_loss += loss.item()
        if i % 200 == 199:
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 200))
            running_loss = 0.0

print('Finished Training')

# Test the network
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        inputs, labels = data[0].to(device), data[1].to(device)
        outputs = net(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 *  correct / total))


[1,   200] loss: 0.971
[1,   400] loss: 0.170
[1,   600] loss: 0.125
[1,   800] loss: 0.090
[2,   200] loss: 0.074
[2,   400] loss: 0.067
[2,   600] loss: 0.055
[2,   800] loss: 0.059
[3,   200] loss: 0.038
[3,   400] loss: 0.045
[3,   600] loss: 0.047
[3,   800] loss: 0.044
[4,   200] loss: 0.034
[4,   400] loss: 0.040
[4,   600] loss: 0.038
[4,   800] loss: 0.035
[5,   200] loss: 0.027
[5,   400] loss: 0.030
[5,   600] loss: 0.032
[5,   800] loss: 0.032
[6,   200] loss: 0.020
[6,   400] loss: 0.026
[6,   600] loss: 0.028
[6,   800] loss: 0.027
[7,   200] loss: 0.019
[7,   400] loss: 0.020
[7,   600] loss: 0.023
[7,   800] loss: 0.021
[8,   200] loss: 0.016
[8,   400] loss: 0.017
[8,   600] loss: 0.020
[8,   800] loss: 0.023
[9,   200] loss: 0.018
[9,   400] loss: 0.015
[9,   600] loss: 0.017
[9,   800] loss: 0.015
[10,   200] loss: 0.009
[10,   400] loss: 0.013
[10,   600] loss: 0.010
[10,   800] loss: 0.012
Finished Training


In [3]:
print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 *  correct / total))



Accuracy of the network on the 10000 test images: 99 %
